In [4]:
#pip install pulp---Only run this if your system doesn have pulp installed in it.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 82.3 MB/s eta 0:00:00


In [5]:
#This cell is the inital data loading cell.
#Consider running "pip install pulp" if pulp is not already installed in your pc
import pulp
#Confiuring our Resources and Constraints
sites = ['S1', 'S2', 'S3']
sources = ['Grid', 'Wind', 'Solar']

max_cap = {'S1':40, 'S2':30, 'S3':30}
con_cost = {'S1':8_000_000, 'S2':6_000_000, 'S3':7_000_000}
jobs = {'S1':4, 'S2':12, 'S3':8}
dev = {'S1':1.00, 'S2':1.30, 'S3':1.10}

max_avail = {'Grid':60, 'Wind':25, 'Solar':15}
pwr_cost = {'Grid':80, 'Wind':95, 'Solar':110}
co2 = {'Grid':350, 'Wind':15, 'Solar':30}

demand = 50
hpy = 8760

def print_results(label, x_vars, cost, emissions, social):
    print(f"\n--- {label} Results ---")
    for i in sites:
        for s in sources:
            val = x_vars[(i,s)].varValue if hasattr(x_vars[(i,s)], 'varValue') else x_vars[(i,s)]
            if val > 1e-2:
                print(f"  {i} ({s}): {val:.2f} MW")
    print(f"Total Cost:        $ {cost:,.0f}")
    print(f"Total Emissions:   {emissions:,.0f} kg CO₂")
    print(f"Total Social Ben.: {social:.1f} job-units")


In [7]:
#Method 1: Aggregated (Weighted Sum) Method
prob = pulp.LpProblem("AggWeighted", pulp.LpMinimize)
x = pulp.LpVariable.dicts("x", ((i,s) for i in sites for s in sources), lowBound=0)

cost_expr = pulp.lpSum([con_cost[i]*x[(i,s)] + pwr_cost[s]*x[(i,s)]*hpy for i in sites for s in sources])
emiss_expr = pulp.lpSum([co2[s]*x[(i,s)]*hpy for i in sites for s in sources])
soc_expr = pulp.lpSum([x[(i,s)]*jobs[i]*dev[i] for i in sites for s in sources])

cost_norm = cost_expr/1e9
emiss_norm = emiss_expr/1e7
soc_norm = soc_expr/1000

prob += 0.4*cost_norm + 0.4*emiss_norm - 0.2*soc_norm

prob += pulp.lpSum([x[(i,s)] for i in sites for s in sources]) == demand
for i in sites:
    prob += pulp.lpSum([x[(i,s)] for s in sources]) <= max_cap[i]
for s in sources:
    prob += pulp.lpSum([x[(i,s)] for i in sites]) <= max_avail[s]

prob.solve()

print_results("Aggregated (Weighted Sum)",
    x,
    pulp.value(cost_expr),
    pulp.value(emiss_expr),
    pulp.value(soc_expr)
)



--- Aggregated (Weighted Sum) Results ---
  S2 (Grid): 5.00 MW
  S2 (Wind): 25.00 MW
  S3 (Grid): 5.00 MW
  S3 (Solar): 15.00 MW
Total Cost:        $ 362,267,000
Total Emissions:   37,887,000 kg CO₂
Total Social Ben.: 644.0 job-units


In [8]:
#Method 2: (B) Weighted Min-Max Method
ref_cost = 450_000_000
ref_emiss = 70_000_000
ref_soc = 780

dev_var = pulp.LpVariable("max_dev", lowBound=0)
prob2 = pulp.LpProblem("MinMaxDev", pulp.LpMinimize)
x2 = pulp.LpVariable.dicts("x2", ((i,s) for i in sites for s in sources), lowBound=0)

cost2 = pulp.lpSum([con_cost[i]*x2[(i,s)] + pwr_cost[s]*x2[(i,s)]*hpy for i in sites for s in sources])
emiss2 = pulp.lpSum([co2[s]*x2[(i,s)]*hpy for i in sites for s in sources])
soc2 = pulp.lpSum([x2[(i,s)]*jobs[i]*dev[i] for i in sites for s in sources])

dev_cost = (cost2-ref_cost)/ref_cost
dev_emiss = (emiss2-ref_emiss)/ref_emiss
dev_soc = (ref_soc-soc2)/ref_soc

prob2 += dev_var
prob2 += dev_cost <= dev_var
prob2 += dev_emiss <= dev_var
prob2 += dev_soc <= dev_var

prob2 += pulp.lpSum([x2[(i,s)] for i in sites for s in sources]) == demand
for i in sites:
    prob2 += pulp.lpSum([x2[(i,s)] for s in sources]) <= max_cap[i]
for s in sources:
    prob2 += pulp.lpSum([x2[(i,s)] for i in sites]) <= max_avail[s]

prob2.solve()

print_results("Weighted Min-Max",
    x2,
    pulp.value(cost2),
    pulp.value(emiss2),
    pulp.value(soc2)
)



--- Weighted Min-Max Results ---
  S2 (Grid): 10.00 MW
  S2 (Wind): 5.00 MW
  S2 (Solar): 15.00 MW
  S3 (Wind): 20.00 MW
Total Cost:        $ 362,267,000
Total Emissions:   37,887,000 kg CO₂
Total Social Ben.: 644.0 job-units


In [9]:
#Method 3: 𝜖-Constrained Method
emiss_limit = 100_000_000
prob3 = pulp.LpProblem("EpsilonConstrained", pulp.LpMinimize)
x3 = pulp.LpVariable.dicts("x3", ((i,s) for i in sites for s in sources), lowBound=0)

cost3 = pulp.lpSum([con_cost[i]*x3[(i,s)] + pwr_cost[s]*x3[(i,s)]*hpy for i in sites for s in sources])
emiss3 = pulp.lpSum([co2[s]*x3[(i,s)]*hpy for i in sites for s in sources])
soc3 = pulp.lpSum([x3[(i,s)]*jobs[i]*dev[i] for i in sites for s in sources])

prob3 += cost3
prob3 += emiss3 <= emiss_limit
prob3 += pulp.lpSum([x3[(i,s)] for i in sites for s in sources]) == demand
for i in sites:
    prob3 += pulp.lpSum([x3[(i,s)] for s in sources]) <= max_cap[i]
for s in sources:
    prob3 += pulp.lpSum([x3[(i,s)] for i in sites]) <= max_avail[s]

prob3.solve()

print_results("Epsilon-Constrained",
    x3,
    pulp.value(cost3),
    pulp.value(emiss3),
    pulp.value(soc3)
)



--- Epsilon-Constrained Results ---
  S2 (Grid): 30.00 MW
  S3 (Grid): 1.84 MW
  S3 (Wind): 18.16 MW
Total Cost:        $ 357,426,570
Total Emissions:   100,000,000 kg CO₂
Total Social Ben.: 644.0 job-units


In [10]:
#Method 4: Lexicographic Method
# Step 1: Minimise Cost
prob4a = pulp.LpProblem("Lex1", pulp.LpMinimize)
x4a = pulp.LpVariable.dicts("x4a", ((i,s) for i in sites for s in sources), lowBound=0)
cost4a = pulp.lpSum([con_cost[i]*x4a[(i,s)] + pwr_cost[s]*x4a[(i,s)]*hpy for i in sites for s in sources])
prob4a += cost4a
prob4a += pulp.lpSum([x4a[(i,s)] for i in sites for s in sources]) == demand
for i in sites:
    prob4a += pulp.lpSum([x4a[(i,s)] for s in sources]) <= max_cap[i]
for s in sources:
    prob4a += pulp.lpSum([x4a[(i,s)] for i in sites]) <= max_avail[s]
prob4a.solve()
min_cost = pulp.value(cost4a)

# Step 2: Minimise Emissions, Cost ≤ 1% above min
prob4b = pulp.LpProblem("Lex2", pulp.LpMinimize)
x4b = pulp.LpVariable.dicts("x4b", ((i,s) for i in sites for s in sources), lowBound=0)
cost4b = pulp.lpSum([con_cost[i]*x4b[(i,s)] + pwr_cost[s]*x4b[(i,s)]*hpy for i in sites for s in sources])
emiss4b = pulp.lpSum([co2[s]*x4b[(i,s)]*hpy for i in sites for s in sources])
soc4b = pulp.lpSum([x4b[(i,s)]*jobs[i]*dev[i] for i in sites for s in sources])
prob4b += emiss4b
prob4b += cost4b <= min_cost * 1.01
prob4b += pulp.lpSum([x4b[(i,s)] for i in sites for s in sources]) == demand
for i in sites:
    prob4b += pulp.lpSum([x4b[(i,s)] for s in sources]) <= max_cap[i]
for s in sources:
    prob4b += pulp.lpSum([x4b[(i,s)] for i in sites]) <= max_avail[s]
prob4b.solve()

print_results("Lexicographic",
    x4b,
    pulp.value(cost4b),
    pulp.value(emiss4b),
    pulp.value(soc4b)
)



--- Lexicographic Results ---
  S2 (Grid): 23.99 MW
  S2 (Wind): 6.01 MW
  S3 (Wind): 18.99 MW
  S3 (Solar): 1.01 MW
Total Cost:        $ 358,590,407
Total Emissions:   77,104,068 kg CO₂
Total Social Ben.: 644.0 job-units


In [11]:
#Method 5: Goal Programming
prob5 = pulp.LpProblem("GoalProg", pulp.LpMinimize)
x5 = pulp.LpVariable.dicts("x5", ((i,s) for i in sites for s in sources), lowBound=0)
cost5 = pulp.lpSum([con_cost[i]*x5[(i,s)] + pwr_cost[s]*x5[(i,s)]*hpy for i in sites for s in sources])
emiss5 = pulp.lpSum([co2[s]*x5[(i,s)]*hpy for i in sites for s in sources])
soc5 = pulp.lpSum([x5[(i,s)]*jobs[i]*dev[i] for i in sites for s in sources])

cost_goal = 560_000_000
emiss_goal = 120_000_000
soc_goal = 700

d_cost_plus = pulp.LpVariable('d_cost_plus', lowBound=0)
d_emiss_plus = pulp.LpVariable('d_emiss_plus', lowBound=0)
d_soc_minus = pulp.LpVariable('d_soc_minus', lowBound=0)

prob5 += d_cost_plus/1e7 + d_emiss_plus/1e6 + d_soc_minus/10

prob5 += cost5 - cost_goal <= d_cost_plus
prob5 += emiss5 - emiss_goal <= d_emiss_plus
prob5 += soc_goal - soc5 <= d_soc_minus

prob5 += pulp.lpSum([x5[(i,s)] for i in sites for s in sources]) == demand
for i in sites:
    prob5 += pulp.lpSum([x5[(i,s)] for s in sources]) <= max_cap[i]
for s in sources:
    prob5 += pulp.lpSum([x5[(i,s)] for i in sites]) <= max_avail[s]

prob5.solve()

print_results("Goal Programming",
    x5,
    pulp.value(cost5),
    pulp.value(emiss5),
    pulp.value(soc5)
)



--- Goal Programming Results ---
  S2 (Grid): 30.00 MW
  S3 (Grid): 8.65 MW
  S3 (Wind): 11.35 MW
Total Cost:        $ 356,531,047
Total Emissions:   120,000,000 kg CO₂
Total Social Ben.: 644.0 job-units
